# Input module examples

This is not yet perfect, but is getting there. The dataframes which are returned are designed to reflect MAGICC's underlying data input format. When we have pint with units (waiting on https://github.com/hgrecco/pint/pull/684) we can use `df.quantify()` and `df.dequantify()` to quickly add/remove units to our dataframes and swiftly do conversions etc. 

In [1]:
from os.path import join
from pprint import pprint

import pandas as pd
import pymagicc
from pymagicc.input import MAGICCInput

import expectexception

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
MAGICC6_DIR = join("..", "pymagicc", "MAGICC6", "run")

In [3]:
"""
temporary sandbox: delete later

DELETE
DELETE
DELETEDELETEDELETE
DELETE
DELETE
DELETE
DELETE
DELETE
"""
mdata = MAGICCInput()
mdata.read(MAGICC6_DIR, "SRESB2.SCEN")
mdata.df

VARIABLE,CO2I,CO2B,CH4,N2O,SOX,CO,NMVOC,NOX,CF4,C2F6,...,C4F10,HFC-23,HFC-32,HFC-43-10,HFC-125,HFC-134a,HFC-143a,HFC-227ea,HFC-245ca,SF6
TODO,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET,...,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET
UNITS,GtC,GtC,MtCH4,MtN2O-N,MtS,MtCO,Mt,MtN,kt,kt,...,kt,kt,kt,kt,kt,kt,kt,kt,kt,kt
REGION,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,...,ALM,ALM,ALM,ALM,ALM,ALM,ALM,ALM,ALM,ALM
YEAR,,,,,,,,,,,,,,,,,,,,,
1990,5.9911,1.1068,309.6788,6.6751,70.9238,879.0190,139.1058,30.9273,15.7893,1.5789,...,0.0,0.0628,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.4743
2000,6.8963,1.0745,322.9364,7.0151,69.0477,877.1473,141.4149,32.0014,12.6000,1.2600,...,0.0,0.1325,0.0000,0.0000,0.0000,5.0000,0.0000,0.0000,0.0000,0.7000
2010,7.9890,0.7952,348.9173,6.2087,65.8975,934.9175,159.2418,36.7336,20.9684,2.0968,...,0.0,0.1970,0.4136,0.1391,1.3673,19.6006,1.0341,1.5078,6.3409,0.8007
2020,9.0177,0.0324,383.9083,6.0598,61.2858,1022.4459,180.2369,42.6658,27.0817,2.7082,...,0.0,0.0884,0.7356,0.2115,2.4347,33.9153,1.8389,2.5867,10.4807,1.0643
2030,10.1494,-0.2520,426.0348,6.1375,60.2944,1110.6278,198.9753,48.8964,34.6437,3.4644,...,0.0,0.0620,1.2133,0.3384,4.0286,55.2820,3.0333,4.1995,17.1051,1.4337
2040,10.9299,-0.2398,466.3648,6.2285,59.0044,1219.6075,213.6447,53.3699,43.6367,4.3637,...,0.0,0.0872,2.0206,0.5614,6.7139,92.0097,5.0514,6.9906,28.5340,2.9929


## Read files

In [4]:
mdata = MAGICCInput()
mdata.read(MAGICC6_DIR, "HISTRCP_CO2I_EMIS.IN")
mdata.head()

VARIABLE CO2I_EMIS                         
TODO           SET                         
UNITS          GtC                         
REGION      R5OECD R5REF R5ASIA R5MAF R5LAM
YEAR                                       
1765         0.003   0.0    0.0   0.0   0.0
1766         0.003   0.0    0.0   0.0   0.0
1767         0.003   0.0    0.0   0.0   0.0
1768         0.003   0.0    0.0   0.0   0.0
1769         0.003   0.0    0.0   0.0   0.0

In [5]:
pprint(mdata.metadata)

{'contact': 'Base year emissions inventories: Steve Smith (ssmith@pnl.gov) and '
            'Jean-Francois Lamarque (Jean-Francois.Lamarque@noaa.gov); RCP '
            '3-PD (IMAGE): Detlef van Vuuren (detlef.vanvuuren@pbl.nl); RCP '
            '4.5 (MiniCAM): Allison Thomson (Allison.Thomson@pnl.gov); RCP 6.0 '
            '(AIM): Toshihiko Masui (masui@nies.go.jp); RCP 8.5 (MESSAGE): '
            'Keywan Riahi (riahi@iiasa.ac.at); Concentrations & Forcing '
            'compilation: Malte Meinshausen (malte.meinshausen@pik-potsdam.de)',
 'data': 'Historical fossil&industrial CO2 (CO2I) Emissions '
         '(HISTRCP_CO2I_EMIS)',
 'header': 'RCPTOOL - MAGICC 6.X DATA FILE \n'
           'VERSION:   ALPHA - FIRST DRAFT - PRIMAP xls file written on 02 Sep '
           '2009, 17:04:37 \n'
           ' \n'
           ' \n'
           'DATA:  Historical fossil&industrial CO2 (CO2I) Emissions '
           '(HISTRCP_CO2I_EMIS) \n'
           'SOURCE:  RCP data as presented on '
         

## Accessors

`MAGICCInput` is built to make data accessing as simple as possible. As we have unique keys for each of our headers, we allow users to select data simply using lists, without specifying what each value should correspond to etc. (essentially we hide away all the pandas fiddly stuff).

Of course the underlying dataframe is always accessible via the `.df` attribute.

In [6]:
mdata['CO2I_EMIS'].head(2)

VARIABLE CO2I_EMIS                         
TODO           SET                         
UNITS          GtC                         
REGION      R5OECD R5REF R5ASIA R5MAF R5LAM
YEAR                                       
1765         0.003   0.0    0.0   0.0   0.0
1766         0.003   0.0    0.0   0.0   0.0

In [7]:
mdata['R5ASIA', 'CO2I_EMIS'].head(2)

VARIABLE,CO2I_EMIS
TODO,SET
UNITS,GtC
REGION,R5ASIA
YEAR,
1765,0.0
1766,0.0


In [8]:
# changing order doesn't matter
mdata['CO2I_EMIS', 'R5ASIA'].head(2)

VARIABLE,CO2I_EMIS
TODO,SET
UNITS,GtC
REGION,R5ASIA
YEAR,
1765,0.0
1766,0.0


In [9]:
%%expect_exception KeyError
# the above doesn't work on the raw dataframe
# because you need to specify all the levels 
# in the right order
mdata.df['R5ASIA', 'CO2I_EMIS']

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
<ipython-input-9-b7d9f3057b51> in <module>()
      2 # because you need to specify all the levels
      3 # in the right order
----> 4 mdata.df['R5ASIA', 'CO2I_EMIS']

~/Documents/AGCEC/MCastle/pymagicc/venv/lib/python3.6/site-packages/pandas/core/frame.py in __getitem__(self, key)
   2684             return self._getitem_frame(key)
   2685         elif is_mi_columns:
-> 2686             return self._getitem_multilevel(key)
   2687         else:
   2688             return self._getitem_column(key)

~/Documents/AGCEC/MCastle/pymagicc/venv/lib/python3.6/site-packages/pandas/core/frame.py in _getitem_multilevel(self, key)
   2728 
   2729     def _getitem_multilevel(self, key):
-> 2730         loc = self.columns.get_loc(key)
   2731         if isinstance(loc, (slice, Series, np.ndarray, Index)):
   2732             new_columns = self.colum

In [10]:
# can also put index values in
mdata[1956, 'CO2I_EMIS', 'R5ASIA'].head(2)

VARIABLE,CO2I_EMIS
TODO,SET
UNITS,GtC
REGION,R5ASIA
YEAR,
1956,0.110425


In [11]:
%%expect_exception KeyError
# although slicing fails, one for the to do list...
mdata[1956:1970, 'CO2I_EMIS', 'R5ASIA'].head(2)

TypeError: (slice(1956, 1970, None), 'CO2I_EMIS', 'R5ASIA')

In [12]:
mdata[1998]

VARIABLE CO2I_EMIS                                        
TODO           SET                                        
UNITS          GtC                                        
REGION      R5OECD     R5REF    R5ASIA     R5MAF     R5LAM
YEAR                                                      
1998      3.184738  0.853781  1.648214  0.522509  0.378757

### Reading SCEN files

We can also read SCEN files. These are particularly tricky as the underlying data format is unique.

In [13]:
mdata.read(MAGICC6_DIR, "RCP45.SCEN")
mdata.df.head()

VARIABLE,CO2I,CO2B,CH4,N2O,SOX,CO,NMVOC,NOX,BC,OC,...,C6F14,HFC23,HFC32,HFC4310,HFC125,HFC134A,HFC143A,HFC227EA,HFC245FA,SF6
TODO,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET,...,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET
UNITS,GtC,GtC,MtCH4,MtN2O-N,MtS,MtCO,Mt,MtN,Mt,Mt,...,kt,kt,kt,kt,kt,kt,kt,kt,kt,kt
REGION,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,...,BUNKERS,BUNKERS,BUNKERS,BUNKERS,BUNKERS,BUNKERS,BUNKERS,BUNKERS,BUNKERS,BUNKERS
YEAR,,,,,,,,,,,,,,,,,,,,,
2000,6.735,1.1488,300.2070,7.4567,53.8413,1068.0009,210.6230,38.1623,7.8048,35.5434,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001,6.896,1.1320,303.4093,7.5029,54.4192,1066.7447,211.5938,38.2888,7.8946,35.7143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002,6.949,1.2317,306.5787,7.5487,54.9960,1065.4692,212.5632,38.4153,7.9841,35.8846,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2003,7.286,1.2256,309.7165,7.5942,55.5716,1064.1742,213.5311,38.5418,8.0734,36.0543,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2004,7.672,1.2428,312.8241,7.6394,56.1461,1062.8596,214.4977,38.6683,8.1624,36.2233,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
mdata["BC", "R5ASIA"].head()

VARIABLE,BC
TODO,SET
UNITS,Mt
REGION,R5ASIA
YEAR,
2000,3.0468
2001,3.1090
2002,3.1818
2003,3.2546
2004,3.3274


## Writing files

Once you have your data in the format as above, writing files is trivial. 

In [15]:
mdata.read(MAGICC6_DIR, "HISTRCP_CO2I_EMIS.IN")
mdata.write("HISTEXAMPLE_CO2I_EMIS.IN")

mdata.read(MAGICC6_DIR, "RCP45.SCEN")
mdata.write("SCENEXAMPLE.SCEN", filepath='.')

However, note that the format to write in is determined by the filename. The rules for this are not yet documented (TODO!) but the errors are fairly clear. Hence you can't just any old filename, it has to follow MAGICC's (somewhat cryptic) internal conventions.

In [16]:
%%expect_exception ValueError
mdata.write("histexample.txt")

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-16-bd740518740c> in <module>()
----> 1 mdata.write("histexample.txt")

~/Documents/AGCEC/MCastle/pymagicc/pymagicc/input.py in write(self, filename, filepath)
    878         TODO: Implement writing to disk
    879         """
--> 880         writer = _get_writer(filename)
    881         writer.write(self, filename, filepath)

~/Documents/AGCEC/MCastle/pymagicc/pymagicc/input.py in _get_writer(fname)
    753 
    754 def _get_writer(fname):
--> 755     return determine_tool(fname, _fname_writer_regex_map, "Writer")()
    756 
    757 

~/Documents/AGCEC/MCastle/pymagicc/pymagicc/input.py in determine_tool(fname, regexp_map, tool_to_find)
    718             return regexp_map[fname_regex]
    719 
--> 720     raise ValueError("Couldn't find appropriate {} for {}".format(tool_to_find, fname))
    721 
    722 

ValueError: